# Module 6: Evaluating What Actually Matters

**Goal:** Choose the right metric for your business problem

**Time:** ~20 minutes

**What you'll do:**
1. Understand precision vs recall tradeoff
2. Pick thresholds based on business constraints
3. Build a cost-sensitive evaluation
4. Check if your probabilities are calibrated

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score, 
    precision_recall_curve, roc_curve
)
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Load and prepare data
df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/data/streamcart_customers.csv')

features = ['tenure_months', 'logins_last_7d', 'logins_last_30d',
            'support_tickets_last_30d', 'items_skipped_last_3_boxes', 'nps_score']

X = df[features].fillna(-1)
y = df['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = lgb.LGBMClassifier(n_estimators=100, max_depth=5, random_state=42, verbose=-1)
model.fit(X_train, y_train)
y_proba = model.predict_proba(X_test)[:, 1]

print(f"Test set: {len(y_test):,} customers")
print(f"Churn rate: {y_test.mean():.1%}")

---

## Part 1: The Precision-Recall Tradeoff

You can't maximize both. Understanding WHY is crucial.

In [ ]:
# Calculate precision and recall at different thresholds
thresholds = [0.05, 0.10, 0.15, 0.20, 0.30, 0.50]

print("=== Threshold Impact ===")
print(f"{'Threshold':<12} {'Precision':<12} {'Recall':<12} {'Flagged %':<12}")
print("-" * 48)

for thresh in thresholds:
    y_pred = (y_proba >= thresh).astype(int)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    flagged = y_pred.mean()
    print(f"{thresh:<12.2f} {prec:<12.1%} {rec:<12.1%} {flagged:<12.1%}")

In [ ]:
# Visualize the tradeoff
precision, recall, thresh = precision_recall_curve(y_test, y_proba)

plt.figure(figsize=(10, 5))
plt.plot(recall, precision, 'b-', linewidth=2)
plt.xlabel('Recall (% of churners found)')
plt.ylabel('Precision (% correct among flagged)')
plt.title('Precision-Recall Curve')
plt.grid(True, alpha=0.3)

# Mark key points
plt.scatter([0.5], [precision[np.argmin(np.abs(recall - 0.5))]], 
            color='red', s=100, zorder=5, label='50% recall')
plt.scatter([0.8], [precision[np.argmin(np.abs(recall - 0.8))]], 
            color='green', s=100, zorder=5, label='80% recall')
plt.legend()
plt.show()

print("\n💡 Moving right (more recall) = precision drops")
print("   You can find more churners, but with more false positives.")

### Key Insight

The tradeoff exists because:
- **High threshold**: Only flag very confident predictions → miss some churners (low recall)
- **Low threshold**: Flag more customers → include non-churners (low precision)

---

## Part 2: Choosing a Threshold

The right threshold depends on your BUSINESS constraint, not a magic number.

In [ ]:
# Scenario 1: Limited capacity (500 calls/week)
# We want: top 500 predictions

capacity = 500

# Find threshold that gives us ~500 predictions
sorted_probs = np.sort(y_proba)[::-1]
threshold_for_capacity = sorted_probs[min(capacity, len(sorted_probs)-1)]

y_pred_cap = (y_proba >= threshold_for_capacity).astype(int)

print(f"=== Capacity-Based Threshold ===")
print(f"Capacity: {capacity} calls/week")
print(f"Threshold: {threshold_for_capacity:.3f}")
print(f"Customers flagged: {y_pred_cap.sum()}")
print(f"Precision: {precision_score(y_test, y_pred_cap):.1%}")
print(f"Recall: {recall_score(y_test, y_pred_cap):.1%}")

In [ ]:
# Scenario 2: Must catch 80% of churners (high recall required)
# Use case: Fraud detection, safety alerts

target_recall = 0.80

# Find threshold that achieves target recall
for t in np.arange(0.01, 0.50, 0.01):
    y_pred_t = (y_proba >= t).astype(int)
    rec = recall_score(y_test, y_pred_t)
    if rec >= target_recall:
        threshold_for_recall = t
        break

y_pred_recall = (y_proba >= threshold_for_recall).astype(int)

print(f"=== Recall-Based Threshold ===")
print(f"Target recall: {target_recall:.0%}")
print(f"Threshold: {threshold_for_recall:.3f}")
print(f"Customers flagged: {y_pred_recall.sum()} ({y_pred_recall.mean():.1%})")
print(f"Precision: {precision_score(y_test, y_pred_recall):.1%}")
print(f"Recall: {recall_score(y_test, y_pred_recall):.1%}")

---

## Part 3: Cost-Based Evaluation

The real question: What's the BUSINESS VALUE of the model?

In [ ]:
# Define business costs
COST_PER_CALL = 15          # Cost to call a customer
VALUE_SAVED_CHURNER = 200   # Value of preventing a churn (via retention offer)
RETENTION_SUCCESS_RATE = 0.3  # 30% of called churners are saved

# TODO: Calculate expected value at different thresholds
#
# For each threshold:
#   - How many customers flagged? (cost = flagged * COST_PER_CALL)
#   - How many true positives? (value = TP * RETENTION_SUCCESS_RATE * VALUE_SAVED_CHURNER)
#   - Net value = value - cost

results = []
for thresh in np.arange(0.05, 0.50, 0.02):
    y_pred = (y_proba >= thresh).astype(int)
    
    flagged = y_pred.sum()
    true_positives = ((y_pred == 1) & (y_test == 1)).sum()
    
    cost = flagged * COST_PER_CALL
    value = true_positives * RETENTION_SUCCESS_RATE * VALUE_SAVED_CHURNER
    net = value - cost
    
    results.append({
        'threshold': thresh,
        'flagged': flagged,
        'true_positives': true_positives,
        'cost': cost,
        'value': value,
        'net_value': net
    })

results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df['net_value'].idxmax()]

print("=== Cost-Based Optimization ===")
print(f"\nOptimal threshold: {best_row['threshold']:.2f}")
print(f"Customers to call: {best_row['flagged']:.0f}")
print(f"True churners caught: {best_row['true_positives']:.0f}")
print(f"Expected saves: {best_row['true_positives'] * RETENTION_SUCCESS_RATE:.0f}")
print(f"\nCost: ${best_row['cost']:,.0f}")
print(f"Value: ${best_row['value']:,.0f}")
print(f"Net value: ${best_row['net_value']:,.0f}")

In [ ]:
# Visualize net value by threshold
plt.figure(figsize=(10, 5))
plt.plot(results_df['threshold'], results_df['net_value'], 'g-', linewidth=2)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.axvline(x=best_row['threshold'], color='blue', linestyle='--', 
            label=f'Optimal ({best_row["threshold"]:.2f})')
plt.xlabel('Threshold')
plt.ylabel('Net Value ($)')
plt.title('Business Value by Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 4: Calibration - Do Probabilities Mean What They Say?

If the model says "30% churn probability", does ~30% of those customers actually churn?

In [ ]:
# Check calibration
prob_true, prob_pred = calibration_curve(y_test, y_proba, n_bins=10)

plt.figure(figsize=(8, 8))
plt.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
plt.plot(prob_pred, prob_true, 'bo-', label='Our model')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives (actual churn rate)')
plt.title('Calibration Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Interpretation:")
print("  - Points above the line: model UNDERestimates risk")
print("  - Points below the line: model OVERestimates risk")

In [ ]:
# Quantify calibration by bucket
print("=== Calibration by Probability Bucket ===")
print(f"{'Predicted Range':<20} {'Actual Rate':<15} {'Customers':<12}")
print("-" * 47)

for i in range(10):
    lower = i * 0.1
    upper = (i + 1) * 0.1
    mask = (y_proba >= lower) & (y_proba < upper)
    if mask.sum() > 0:
        actual_rate = y_test[mask].mean()
        expected_rate = (lower + upper) / 2
        count = mask.sum()
        print(f"{lower:.1f} - {upper:.1f}          {actual_rate:>10.1%}     {count:>8}")

---

## Part 5: Which Metric for Which Situation?

In [ ]:
guidance = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  SITUATION                     │  PRIMARY METRIC         │  WHY             │
├─────────────────────────────────────────────────────────────────────────────┤
│  Limited capacity              │  Precision@K            │  Maximize ROI    │
│  (500 calls, 100 reviews)      │                         │  per action      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Can't miss positives          │  Recall + Precision     │  Catch all       │
│  (fraud, safety, disease)      │  at target recall       │  bad cases       │
├─────────────────────────────────────────────────────────────────────────────┤
│  Costs are quantifiable        │  Cost-weighted          │  Maximize        │
│  (known $ values)              │  net value              │  profit          │
├─────────────────────────────────────────────────────────────────────────────┤
│  Comparing models              │  AUC                    │  Ranking         │
│  (which is better overall)     │                         │  ability         │
├─────────────────────────────────────────────────────────────────────────────┤
│  Probabilistic decisions       │  Calibration +          │  Trust the       │
│  (expected value calcs)        │  Brier score            │  numbers         │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(guidance)

---

## 📝 Final Exercise: Explain It

The finance team asks: "What's the ROI of this model? How do we know it's worth investing in?"

Write a 4-5 sentence response using the cost analysis above.

In [ ]:
# Write your response:

finance_response = """
YOUR RESPONSE HERE
"""

print(finance_response)

---

## ✅ Module 6 Complete!

**What you learned:**
- The precision-recall tradeoff and why it exists
- How to choose thresholds based on business constraints
- How to evaluate with business costs/values
- What calibration means and why it matters

In [ ]:
# Summary
print("=== Module 6 Summary ===")
print(f"\nModel AUC: {roc_auc_score(y_test, y_proba):.3f}")
print(f"\nBest threshold (cost-optimized): {best_row['threshold']:.2f}")
print(f"Expected net value: ${best_row['net_value']:,.0f}")

**Next:** [Module 7: Finding Natural Groups →](./module_07_clustering.ipynb)